<a href="https://colab.research.google.com/github/SunnyORZ030/255---AutoGluon/blob/main/colabs/05_kaggle_california_official.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
os.environ["CUDA_VISIBLE_DEVICES"] = ""   # 先用 CPU 即可
!{sys.executable} -m pip install -U pip setuptools wheel
!{sys.executable} -m pip install -U autogluon.tabular autogluon.core autogluon.common autogluon.multimodal kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.4 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at

In [2]:
from google.colab import files
import os, pathlib, shutil, stat

uploaded = files.upload()   # 選剛下載的 kaggle.json
home = pathlib.Path.home()
kag_dir = home/".kaggle"; kag_dir.mkdir(exist_ok=True)
shutil.move("kaggle.json", kag_dir/"kaggle.json")
os.chmod(kag_dir/"kaggle.json", stat.S_IRUSR | stat.S_IWUSR)  # 權限 600
print("kaggle.json ready at", kag_dir/"kaggle.json")

Saving kaggle.json to kaggle.json
kaggle.json ready at /root/.kaggle/kaggle.json


In [3]:
# 列出含 "california housing" 關鍵字的資料集（看到清單就表示 API 正常）
!kaggle datasets list -s "california housing" | head -n 20

ref                                                        title                                                     size  lastUpdated                 downloadCount  voteCount  usabilityRating  
---------------------------------------------------------  --------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
camnugent/california-housing-prices                        California Housing Prices                               409382  2017-11-24 03:14:59.677000         260244       1504  0.85294116       
harrywang/housing                                          California Housing Data (1990)                          409747  2018-05-10 15:56:31.027000          19948        183  1.0              
dhirajnirne/california-housing-data                        California Housing Data                                 409382  2021-05-15 21:21:05.067000           4841         29  1.0              
fedesoriano/california-ho

In [4]:
!kaggle datasets download -d harrywang/housing -p /content/data -q
!unzip -o /content/data/housing.zip -d /content/data >/dev/null 2>&1
!ls -lh /content/data

Dataset URL: https://www.kaggle.com/datasets/harrywang/housing
License(s): copyright-authors
total 1.8M
-rw-r--r-- 1 root root  556 Sep 30  2019 anscombe.csv
-rw-r--r-- 1 root root 1.4M Sep 30  2019 housing.csv
-rw-r--r-- 1 root root 401K Sep 30  2019 housing.zip


In [5]:
import pandas as pd
from autogluon.tabular import TabularDataset

csv_path = "/content/data/housing.csv"
df = TabularDataset(csv_path)
print("Shape:", df.shape)
print("Columns:", list(df.columns)[:10], "...")

# 自動偵測常見目標欄位名稱
candidates = ["median_house_value", "MedHouseVal", "MedianHouseValue"]
label = next((c for c in candidates if c in df.columns), None)
if not label:
    print("I don’t know which column is the label. Available columns:", list(df.columns))
    raise SystemExit("請告訴我哪一欄是 y（例如 median_house_value），我再幫你改程式。")

print("Label =", label)

Shape: (20640, 10)
Columns: ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity'] ...
Label = median_house_value


In [6]:
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularPredictor

# 打亂後切分
df_shuf = df.sample(frac=1.0, random_state=42)
train_df, test_df = train_test_split(df_shuf, test_size=0.2, random_state=42)

metric = "rmse"  # 回歸常用 RMSE
fast_hp = {
    "GBM": {}, "CAT": {}, "XGB": {}, "RF": {}, "XT": {}, "LR": {},
    "NN_TORCH": [], "AG_AUTOMM": []      # 關掉慢的 NN/多模態
}

predictor = TabularPredictor(
    label=label, eval_metric=metric, problem_type="regression"
).fit(
    train_data=train_df,
    presets="medium_quality_faster_train",
    time_limit=180,                      # 視情況 120~300
    hyperparameters=fast_hp,
    num_bag_folds=0, num_stack_levels=0,
    ag_args_fit={"num_gpus": 0}         # 強制 CPU
)

# 確認訓練出的模型
preview = predictor.leaderboard(silent=True)[["model","score_val"]]
preview.head(10)

No path specified. Models will be saved in: "AutogluonModels/ag-20251015_035920"
Preset alias specified: 'medium_quality_faster_train' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Memory Avail:       11.38 GB / 12.67 GB (89.8%)
Disk Space Avail:   62.51 GB / 107.72 GB (58.0%)
Presets specified: ['medium_quality_faster_train']
Beginning AutoGluon training ... Time limit = 180s
AutoGluon will save models to "/content/AutogluonModels/ag-20251015_035920"
Train Data Rows:    16512
Train Data Columns: 9
Label Column:       median_house_value
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11634.99 MB
	Train Dat

[1000]	valid_set's rmse: 46459


	-46397.0183	 = Validation score   (-root_mean_squared_error)
	15.62s	 = Training   runtime
	0.26s	 = Validation runtime
Fitting model: RandomForest ... Training model for up to 163.72s of the 163.72s of remaining time.
	Fitting with cpus=2, gpus=0
	-49924.3982	 = Validation score   (-root_mean_squared_error)
	43.74s	 = Training   runtime
	0.27s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 118.69s of the 118.68s of remaining time.
	Fitting with cpus=1, gpus=0
		`import catboost` failed. A quick tip is to install via `pip install autogluon.tabular[catboost]==1.4.0`.
Fitting model: ExtraTrees ... Training model for up to 118.41s of the 118.40s of remaining time.
	Fitting with cpus=2, gpus=0
	-52920.5473	 = Validation score   (-root_mean_squared_error)
	13.94s	 = Training   runtime
	0.61s	 = Validation runtime
Fitting model: XGBoost ... Training model for up to 100.67s of the 100.66s of remaining time.
	Fitting with cpus=1, gpus=0
	-45819.2128	 = Validation s

,model,score_val
0,WeightedEnsemble_L2,-45223.671398
1,XGBoost,-45819.212752
2,LightGBM,-46397.018314
3,RandomForest,-49924.398200
4,ExtraTrees,-52920.547346
5,LinearModel,-71700.839641


In [7]:
from pathlib import Path
import json, os
import numpy as np
RESULTS = Path("/content/results"); RESULTS.mkdir(parents=True, exist_ok=True)

# 1) Leaderboard（含 test 評分）
lb = predictor.leaderboard(test_df, silent=True)
lb.to_csv(RESULTS/"leaderboard_california_official.csv", index=False)

# 2) Metrics（高層 evaluate；只把數值轉 float）
eval_out = predictor.evaluate(test_df, silent=True)
metrics = eval_out if isinstance(eval_out, dict) else {"score_test": float(eval_out)}
metrics["eval_metric"] = metric

def to_jsonable(v):
    if isinstance(v, (int, float, np.integer, np.floating)): return float(v)
    return v

with open(RESULTS/"metrics_california_official.json", "w") as f:
    json.dump({k: to_jsonable(v) for k, v in metrics.items()}, f, indent=2)

# 3)（加分）特徵重要度
fi = predictor.feature_importance(test_df, subsample_size=10000, silent=True)
fi.reset_index().rename(columns={"index": "feature"}).to_csv(
    RESULTS/"feature_importance_california.csv", index=False
)

print("Saved under /content/results:")
print(*sorted(os.listdir(RESULTS)), sep="\n - ")

/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


Saved under /content/results:
feature_importance_california.csv
 - leaderboard_california_official.csv
 - metrics_california_official.json
